In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Telus_Query ='''


with ADC_customer_base as(

SELECT distinct customer_id,dealer_customer_id,dealer_name,account_type_name,customer_type_name
FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` 
where date(last_updt_ts) = '2022-08-31'
--and account_type_name!='Standalone'
--and customer_type_name='Customer'
and dealer_name='TELUS Communications Inc.'
QUALIFY ROW_NUMBER() OVER (PARTITION BY dealer_customer_id ORDER BY last_updt_ts DESC) = 1
--order by dealer_name,join_date,customer_id
)
,

Telus_customers as
(
select bacct_bus_bacct_num,bacct_billg_acct_id,cust_bus_cust_id,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,prod_instnc_ts,pi_cntrct_end_ts as contract_end_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = "2022-08-31" #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY pi_prod_instnc_stat_ts DESC) = 1
order by cust_bus_cust_id

)


,Telus_internet_customers as 
(

select distinct (customer_id) as cust_id_telus,Telus_internet_flag from `divgpras-pr-579355.ADC_updated.Telus_internet_31Aug2022`

)


, Telus_SMHM_Deact_Sep2022 as 


(


select distinct(CAST (CUST_ID as STRING)) as CUST_ID ,Deact_Flag as Telus_Churn_Flag  

from `divgpras-pr-579355.ADC_updated.Telus_SMHM_Sep2022_Deacts`


)

select * from ADC_customer_base as ADC
inner join Telus_customers as Telus
on ADC.dealer_customer_id=Telus.cust_bus_cust_id
left join Telus_internet_customers
on ADC.customer_id=Telus_internet_customers.cust_id_telus
left join Telus_SMHM_Deact_Sep2022
on ADC.dealer_customer_id=Telus_SMHM_Deact_Sep2022.CUST_ID

'''


In [ ]:
Data_Telus=extract_bq_data(bq_client, sql=Telus_Query)

In [ ]:
Data_Telus.info()

In [ ]:
Data_Telus['Cust_id_1']=Data_Telus['dealer_customer_id'].astype('int64')

In [ ]:
ARPU=pd.read_csv('Telus_SMHM_ARPU_Sep_Apr2022.csv')

In [ ]:
ARPU.info()

In [ ]:
# ARPU['cust_id']=ARPU['cust_id'].astype('O')

In [ ]:
ARPU.head()

In [ ]:
ARPU['ARPU_flag']=1

In [ ]:
Data_Telus[Data_Telus['cust_bus_cust_id']=='100402370']

In [ ]:
Data_Telus_merge=Data_Telus.merge(ARPU,left_on='Cust_id_1',right_on='cust_id',how='left')

In [ ]:
Data_Telus_merge['ARPU_flag'].isna().sum()*100/Data_Telus_merge.shape[0]

In [ ]:
Data_Telus_merge.head()

In [ ]:
pd.DataFrame(Data_Telus_merge.groupby(['account_type_name']).agg(
    
    Customer_count= ('customer_id','nunique'),
    ARPU_Avg=('ffh_amt_0','mean')))